In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

table_data = []

sugang_url = 'https://mportal.cau.ac.kr/std/usk/sUskSif001/search.do'
driver = webdriver.Chrome()
driver.get(sugang_url)
time.sleep(3)

# 2022년 
year_option = driver.find_element(By.XPATH, '//*[@id="sel_year"]/option[2]')
year_option.click()

# 학부 
course_option = driver.find_element(By.XPATH, '//*[@id="sel_course"]')
course_option.click()

for s in range(1, 4):  # 학기
    if s == 1 or s == 3:
        # 학기 선택
        shtm_option = driver.find_element(By.XPATH, '//*[@id="sel_shtm"]/option[' + str(s) + ']')
        shtm_option.click()
        time.sleep(1)  

        for c in range(2, 4):  # 캠퍼스
            # 캠퍼스 선택
            camp_option = driver.find_element(By.XPATH, '//*[@id="sel_camp"]/option[' + str(c) + ']')
            camp_option.click()
            time.sleep(1)  

            for d in range(3, len(driver.find_elements(By.XPATH, '//*[@id="sel_colg"]/option'))+1):
                # 단과대 선택
                colg_option = driver.find_element(By.XPATH, '//*[@id="sel_colg"]/option[' + str(d) + ']')
                colg_option.click()
                time.sleep(1)  

                for i in range(2, len(driver.find_elements(By.XPATH, '//*[@id="sel_sust"]/option'))+1):
                    # 학과(부)/전공/영역 선택
                    sust_option = driver.find_element(By.XPATH, '//*[@id="sel_sust"]/option[' + str(i) + ']')
                    sust_value = sust_option.text.strip()   # df column 추가 위해
                    sust_option.click()
                    time.sleep(0.1)  

                    # Click the 검색 button
                    search_button = driver.find_element(By.XPATH, '//button[contains(text(), "검색")]')
                    search_button.click()
                    time.sleep(1)

                    # table data
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    data = soup.find_all("div", {'class':'sp-grid-data-row'})
                    
                    for row in data:
                        row_data = [span.text.strip() for span in row.find_all('span', class_='ng-binding ng-scope')]
                        row_data.insert(1, sust_value)  # Insert 학과(부)/전공/영역 정보
                        table_data.append(row_data)

# table data as DataFrame
column_names = ['대학', '학과(부)/전공/영역', '개설학과', '학년', '과정', '이수구분', '과목번호-분반', '과목명', '학점-시간', '담당교수', '폐강', '강의시간', '유연학기', '비고', '수업유형']
df = pd.DataFrame(data=table_data, columns=column_names)
print(df)

           대학 학과(부)/전공/영역   개설학과  학년  과정  이수구분   과목번호-분반                과목명  \
0      대학(전체)  외식산업경영연계전공         전체  학사    교양  28703-01              경제와사회   
1      대학(전체)  외식산업경영연계전공         전체  학사    교양  28703-01              경제와사회   
2      대학(전체)  외식산업경영연계전공         전체  학사    교양  28703-02              경제와사회   
3      대학(전체)  외식산업경영연계전공         전체  학사    교양  28703-02              경제와사회   
4      대학(전체)  외식산업경영연계전공         전체  학사    교양  28703-03              경제와사회   
...       ...         ...    ...  ..  ..   ...       ...                ...   
18058  예술공학대학       예술공학부  예술공학부   3  학사  전공필수  56845-02      캡스톤 디자인 프로젝트Ⅰ   
18059  예술공학대학       예술공학부  예술공학부   3  학사  전공필수  56845-03      캡스톤 디자인 프로젝트Ⅰ   
18060  예술공학대학       예술공학부  예술공학부   3  학사    전공  55403-01  피지컬컴퓨팅과제어 (영어A강의)   
18061  예술공학대학       예술공학부  예술공학부   3  학사    전공  55403-02  피지컬컴퓨팅과제어 (영어A강의)   
18062  예술공학대학       예술공학부  예술공학부   4  학사    전공  56084-01        이머시브 미디어 기술   

      학점-시간 담당교수 폐강                                

In [5]:
selected_columns =  ['학과(부)/전공/영역', '학년', '이수구분', '과목번호-분반', '과목명']
new_df = df[selected_columns]
print(new_df)

      학과(부)/전공/영역  학년  이수구분   과목번호-분반                과목명
0      외식산업경영연계전공  전체    교양  28703-01              경제와사회
1      외식산업경영연계전공  전체    교양  28703-01              경제와사회
2      외식산업경영연계전공  전체    교양  28703-02              경제와사회
3      외식산업경영연계전공  전체    교양  28703-02              경제와사회
4      외식산업경영연계전공  전체    교양  28703-03              경제와사회
...           ...  ..   ...       ...                ...
18058       예술공학부   3  전공필수  56845-02      캡스톤 디자인 프로젝트Ⅰ
18059       예술공학부   3  전공필수  56845-03      캡스톤 디자인 프로젝트Ⅰ
18060       예술공학부   3    전공  55403-01  피지컬컴퓨팅과제어 (영어A강의)
18061       예술공학부   3    전공  55403-02  피지컬컴퓨팅과제어 (영어A강의)
18062       예술공학부   4    전공  56084-01        이머시브 미디어 기술

[18063 rows x 5 columns]


In [8]:
new_df.to_csv('C:/Users/황의지/Desktop/hk+/cau_timeschedule.csv', index=False, encoding='utf-8-sig')